<a href="https://colab.research.google.com/github/isabelvrielink/Paralinguistic-feature-extraction-for-audio-deepfake-detection/blob/main/Coding_part_2_(pre_processing_%26_testing_models).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Content
# 1. Pre-process all data in seperate dataframes, labels - embeddings
# 2. Display train set, evaluation set and test set
# 3. log reg architecture
# 4. log reg training
# 5. Performance metrics
# 6. Equal Error Rate (EER)
# 7. Performance

In [ ]:
# install packages
!pip install --upgrade kagglehub
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
!pip install tensorflow-hub
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
!apt-get -qq install -y graphviz && pip install pydot
!pip install cartopy
!pip install pydub
!pip install librosa
!pip install scikit-learn


E: Package 'libfluidsynth1' has no installation candidate
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 7.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for libarchive
  Running setup.py clean for libarchive
Failed to build libarchive
ERROR: Could not build wheels for libarchive, which is required to install pyproject.toml-based projects
     ━━━━━━━━

In [ ]:
# import modules
import kagglehub
import pydot
import cartopy
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import numpy as np
import pandas as pd
import os
from pydub import AudioSegment
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, accuracy_score, precision_score, recall_score, f1_score


In [ ]:
# mount drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


In [ ]:
!pip install pydub
!pip install librosa

In [ ]:
# Pre processing:
# - ran all audio clips into 1 second clips locally (except for the dev set) and uploaded it to drive
# - now: run all 1 second clips through the TRILLsson process

In [ ]:
# check for processing issues
import os
import glob

# Path to your Google Drive directory
drive_dir = '/path/to/newtrain_input'

# Check if the directory exists
if os.path.exists(drive_dir):
    print(f"The directory {drive_dir} exists.")
else:
    print(f"The directory {drive_dir} does not exist.")

# Verify files with the correct extension
files = glob.glob(os.path.join(drive_dir, '*.flac'))
print(f"Found {len(files)} .flac files to process.")
for file in files:
    print(file)


Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/THESIS ISABEL/TRAINING/OUD_newtrain_input/LA_T_3490050.flac
/content/drive/My Drive/THESIS ISABEL/TRAINING/OUD_newtrain_input/LA_T_5112697.flac
/content/drive/My Drive/THESIS ISABEL/TRAINING/OUD_newtrain_input/LA_T_5179598.flac
/content/drive/My Drive/THESIS ISABEL/TRAINING/OUD_newtrain_input/LA_T_8798465.flac
/content/drive/My Drive/THESIS ISABEL/TRAINING/OUD_newtrain_input/LA_T_8651715.flac
/content/drive/My Drive/THESIS ISABEL/TRAINING/OUD_newtrain_input/LA_T_1215743.flac
/content/drive/My Drive/THESIS ISABEL/TRAINING/OUD_newtrain_input/LA_T_8951473.flac
/content/drive/My Drive/THESIS ISABEL/TRAINING/OUD_newtrain_input/LA_T_4414705.flac
/content/drive/My Drive/THESIS ISABEL/TRAINING/OUD_newtrain_input/LA_T_6723039.flac
/content/drive/My Drive/THESIS ISABEL/TRAINING/OUD_newtrain_input/LA_T_7657247.flac
/content/drive/My Drive/THESIS ISABEL/TRAINING/OUD_newtrain_input/LA_T_4564345.flac
/content/drive/My Drive/T

KeyboardInterrupt: 

In [ ]:
# Run training set through TRILLsson
# Code reused from thesis supervisor, Prof. Dr. Eric Postma (Tilburg University)

import tensorflow as tf
import tensorflow_hub as hub
import glob
import numpy as np
import librosa
import os
from datetime import datetime
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# Mount Google Drive if using Google Colab
from google.colab import drive
drive.mount('/content/drive')

model = hub.KerasLayer('https://tfhub.dev/google/trillsson5/1')

seconds = 1

# Store embeddings
embeddings_dir = '/path/to/train_embeddings'
os.makedirs(embeddings_dir, exist_ok=True)

# files directory
files = glob.glob('/path/to/newtrain_input/*.flac')
print(f"Found {len(files)} files to process.")

for filename in files:
  if os.path.isfile(filename):
    head, tail = os.path.split(filename)
    filestem = tail[:-5]
    outfilename = f"{embeddings_dir}/{filestem}.txt"  # Ensure 'T' is preserved

    # Check if 'T' is in the filestem to ensure it's correct
    if 'T' not in filestem:
      print(f"Warning: Expected 'T' in filename but found: {filestem}")

    if not os.path.isfile(outfilename):
      y, sr = librosa.load(filename, sr=16000)
      segment = y[0:seconds * sr]
      if len(segment) / sr >= 1:
        y_mono = librosa.to_mono(segment)
        speech = librosa.util.normalize(y_mono)
        audio_sample = tf.convert_to_tensor(tf.reshape(speech, (1, speech.size)))
        embedding = model(audio_sample)['embedding']
        embedding.shape.assert_is_compatible_with([None, 1024])
        np.savetxt(outfilename, embedding.numpy())

# Check saved files in the specific directory
saved_files = glob.glob(f"{embeddings_dir}/*.txt")
print(f"Saved {len(saved_files)} embedding files.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 25420 files to process.
Saved 25328 embedding files.


In [ ]:
# Run evaluation set through TRILLsson,
# This procedure is done in batches to tackle computational issues for the evaluation set
# File for this is called: evaluation trillsson run
# (Dev set is already ran through TRILLsson and set as input for the logreg preprocessing)
# Code reused from thesis supervisor, Prof. Dr. Eric Postma (Tilburg University)

import tensorflow as tf
import tensorflow_hub as hub
import glob
import numpy as np
import librosa
import os
from datetime import datetime
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# Mount Google Drive if using Google Colab
from google.colab import drive
drive.mount('/content/drive')

model = hub.KerasLayer('https://tfhub.dev/google/trillsson5/1')

seconds = 1

# Store embeddings
embeddings_dir = '/path/to/eval_embeddings'
os.makedirs(embeddings_dir, exist_ok=True)

# files directory
files = glob.glob('/path/to/ASVspoof2019_LA_eval/evalnew_input/*.flac')
print(f"Found {len(files)} files to process.")

for filename in files:
  if os.path.isfile(filename):
    head, tail = os.path.split(filename)
    filestem = tail[:-5]
    outfilename = f"{embeddings_dir}/{filestem}.txt"  # Ensure 'E' is preserved

    # Check if 'E' is in the filestem to ensure it's correct
    if 'E' not in filestem:
      print(f"Warning: Expected 'E' in filename but found: {filestem}")

    if not os.path.isfile(outfilename):
      y, sr = librosa.load(filename, sr=16000)
      segment = y[0:seconds * sr]
      if len(segment) / sr >= 1:
        y_mono = librosa.to_mono(segment)
        speech = librosa.util.normalize(y_mono)
        audio_sample = tf.convert_to_tensor(tf.reshape(speech, (1, speech.size)))
        embedding = model(audio_sample)['embedding']
        embedding.shape.assert_is_compatible_with([None, 1024])
        np.savetxt(outfilename, embedding.numpy())

# Check saved files in the specific directory
saved_files = glob.glob(f"{embeddings_dir}/*.txt")
print(f"Saved {len(saved_files)} embedding files.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 0 files to process.
Saved 0 embedding files.


In [ ]:
# Pre processing, put all embeddings in a dataframe with its corresponding labels

In [ ]:
# Put the training set in a dataframe with its corresponding labels
# Filter out pairs that cannot be made because of duplicates or missing embeddings/labels

# Connect the labels with the embeddings

import os
import numpy as np
import pandas as pd

# Path to both files
embeddings_dir = '/path/to/TRAINING/train_embeddings'
label_file_path = '/path/to/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt'

# Load labels file and put in dict
labels_dict = {}
with open(label_file_path, 'r') as file:
  for line in file:
    parts = line.strip().split()
    file_id = parts[1] + ".txt"
    label = parts[-1]
    labels_dict[file_id] = 1 if label == 'bonafide' else 0

print(f"Loaded {len(labels_dict)} labels")

# Ensure the embeddings directory exists
if not os.path.exists(embeddings_dir):
    print(f"Embeddings directory {embeddings_dir} does not exist.")
else:
    print(f"Embeddings directory exists: {embeddings_dir}")

# List files in embeddings directory
embeddings_files = os.listdir(embeddings_dir)
print(f"Total embeddings files: {len(embeddings_files)}")

# Sample embeddings files
print("Sample embeddings files:", embeddings_files[:10])

# Sample label identifiers
sample_labels = list(labels_dict.keys())[:10]
print("Sample label identifiers:", sample_labels)

# Tracking pairing, debugging purposes
successful_loads = 0
failed_loads = 0

# Filter out duplicates and missing files
matched_files = []
embeddings_dict = {}

for filename in embeddings_files:
  filepath = os.path.join(embeddings_dir, filename)
  if filename in labels_dict:
    try:
      embedding = np.loadtxt(filepath)
      embeddings_dict[filename] = embedding
      matched_files.append(filename)
      successful_loads += 1
    except Exception as e:
      print(f"Failed to load {filename}: {e}")
      failed_loads += 1

print(f"Number of matched files: {len(matched_files)}")
print(f"Loaded {successful_loads} embeddings")
print(f"Failed to load {failed_loads} embeddings")

# Filtered out unmatched labels
filtered_labels_dict = {k: v for k, v in labels_dict.items() if k in matched_files}

print(f"Filtered to {len(filtered_labels_dict)} matched labels")

# Create dataframe
data = {
    'embedding': [embeddings_dict[filename] for filename in matched_files],
    'label': [filtered_labels_dict[filename] for filename in matched_files]}


df_train = pd.DataFrame(data)
print(df_train.head())

Loaded 25380 labels
Embeddings directory exists: /content/drive/My Drive/THESIS ISABEL/TRAINING/train_embeddings
Total embeddings files: 25328
Sample embeddings files: ['LA_T_2185190.txt', 'LA_T_3394803.txt', 'LA_T_4844784.txt', 'LA_T_6550411.txt', 'LA_T_3077161.txt', 'LA_T_9107429.txt', 'LA_T_6795317.txt', 'LA_T_5028868.txt', 'LA_T_2757131.txt', 'LA_T_3286480.txt']
Sample label identifiers: ['LA_T_1138215.txt', 'LA_T_1271820.txt', 'LA_T_1272637.txt', 'LA_T_1276960.txt', 'LA_T_1341447.txt', 'LA_T_1363611.txt', 'LA_T_1596451.txt', 'LA_T_1608170.txt', 'LA_T_1684951.txt', 'LA_T_1699801.txt']
Number of matched files: 25288
Loaded 25288 embeddings
Failed to load 0 embeddings
Filtered to 25288 matched labels
                                           embedding  label
0  [-1.1427929401397705, -1.2910186052322388, -0....      0
1  [-1.3630930185317993, -1.56317138671875, -0.69...      0
2  [-1.0894581079483032, -1.3105745315551758, -0....      0
3  [-1.4806939363479614, -0.5242177248001099, -0

In [ ]:
# save dataframe as csv

file_path = '/path/to//df_train.csv'
df_train.to_csv(file_path, index=False)

print(f'DataFrame saved to {file_path}')

DataFrame saved to /content/drive/My Drive/THESIS ISABEL/Dataframes/df_train.csv


In [ ]:
# save dataframe as pkl
import pandas as pd

pickle_file_path = '/path/to/df_train.pkl'
df_train.to_pickle(pickle_file_path)
print(f'DataFrame saved to {pickle_file_path}')

# Load the DataFrame from the pickle file
df_train_loaded = pd.read_pickle(pickle_file_path)
print(df_train_loaded.head())

DataFrame saved to /content/drive/My Drive/THESIS ISABEL/Dataframes/df_train.pkl
                                           embedding  label
0  [-1.1427929401397705, -1.2910186052322388, -0....      0
1  [-1.3630930185317993, -1.56317138671875, -0.69...      0
2  [-1.0894581079483032, -1.3105745315551758, -0....      0
3  [-1.4806939363479614, -0.5242177248001099, -0....      0
4  [-0.9210010766983032, -0.856410801410675, -0.8...      0


In [ ]:
# Put the evaluation set in a dataframe with its corresponding labels
# Filter out pairs that cannot be made because of duplicates or missing embeddings/labels

import os
import numpy as np
import pandas as pd

# Path to both files
embeddings_dir = '/path/to/eval_embeddings'
label_file_path = '/path/to/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt'

# Load labels file and put in dict
labels_dict = {}
with open(label_file_path, 'r') as file:
  for line in file:
    parts = line.strip().split()
    file_id = parts[1] + ".txt"
    label = parts[-1]
    labels_dict[file_id] = 1 if label == 'bonafide' else 0

print(f"Loaded {len(labels_dict)} labels")

# Ensure the embeddings directory exists
if not os.path.exists(embeddings_dir):
    print(f"Embeddings directory {embeddings_dir} does not exist.")
else:
    print(f"Embeddings directory exists: {embeddings_dir}")

# List files in embeddings directory
embeddings_files = os.listdir(embeddings_dir)
print(f"Total embeddings files: {len(embeddings_files)}")

# Sample embeddings files
print("Sample embeddings files:", embeddings_files[:10])

# Sample label identifiers
sample_labels = list(labels_dict.keys())[:10]
print("Sample label identifiers:", sample_labels)

# Tracking pairing, debugging purposes
successful_loads = 0
failed_loads = 0

# Filter out duplicates and missing files
matched_files = []
embeddings_dict = {}

for filename in embeddings_files:
  filepath = os.path.join(embeddings_dir, filename)
  if filename in labels_dict:
    try:
      embedding = np.loadtxt(filepath)
      embeddings_dict[filename] = embedding
      matched_files.append(filename)
      successful_loads += 1
    except Exception as e:
      print(f"Failed to load {filename}: {e}")
      failed_loads += 1

print(f"Number of matched files: {len(matched_files)}")
print(f"Loaded {successful_loads} embeddings")
print(f"Failed to load {failed_loads} embeddings")

# Filtered out unmatched labels
filtered_labels_dict = {k: v for k, v in labels_dict.items() if k in matched_files}

print(f"Filtered to {len(filtered_labels_dict)} matched labels")

# Create dataframe
data = {
    'embedding': [embeddings_dict[filename] for filename in matched_files],
    'label': [filtered_labels_dict[filename] for filename in matched_files]}


df_eval = pd.DataFrame(data)
print(df_eval.head())

Loaded 71237 labels
Embeddings directory exists: /content/drive/My Drive/THESIS ISABEL/EVAL/eval_embeddings
Total embeddings files: 30342
Sample embeddings files: ['LA_E_4506103.txt', 'LA_E_4199464.txt', 'LA_E_4743278.txt', 'LA_E_4327757.txt', 'LA_E_4403599.txt', 'LA_E_4272880.txt', 'LA_E_4718380.txt', 'LA_E_4749225.txt', 'LA_E_5004499.txt', 'LA_E_4488283.txt']
Sample label identifiers: ['LA_E_2834763.txt', 'LA_E_8877452.txt', 'LA_E_6828287.txt', 'LA_E_6977360.txt', 'LA_E_5932896.txt', 'LA_E_5849185.txt', 'LA_E_6163791.txt', 'LA_E_4581379.txt', 'LA_E_8814547.txt', 'LA_E_9157999.txt']
Number of matched files: 30341
Loaded 30341 embeddings
Failed to load 0 embeddings
Filtered to 30341 matched labels
                                           embedding  label
0  [-0.6744308471679688, -1.00356125831604, -1.24...      0
1  [-1.1688693761825562, -1.836666464805603, -0.8...      0
2  [-1.7681496143341064, -1.2730071544647217, -0....      0
3  [-1.0557725429534912, -0.8026940226554871, -0.... 

In [ ]:
# save dataframe as csv
file_path = '/path/to//df_eval.csv'
df_eval.to_csv(file_path, index=False)

print(f'DataFrame saved to {file_path}')

DataFrame saved to /content/drive/My Drive/THESIS ISABEL/Dataframes/df_eval.csv


In [ ]:
# save dataframe as pkl
import pandas as pd

pickle_file_path = '/path/to/df_eval.pkl'
df_eval.to_pickle(pickle_file_path)
print(f'DataFrame saved to {pickle_file_path}')

# Load the DataFrame from the pickle file
df_eval_loaded = pd.read_pickle(pickle_file_path)
print(df_eval_loaded.head())

DataFrame saved to /content/drive/My Drive/THESIS ISABEL/Dataframes/df_eval.pkl
                                           embedding  label
0  [-0.6744308471679688, -1.00356125831604, -1.24...      0
1  [-1.1688693761825562, -1.836666464805603, -0.8...      0
2  [-1.7681496143341064, -1.2730071544647217, -0....      0
3  [-1.0557725429534912, -0.8026940226554871, -0....      0
4  [-1.795451283454895, -0.9414591789245605, -0.7...      0


In [ ]:
# Put the dev set in a dataframe with its corresponding labels
# Filter out pairs that cannot be made because of duplicates or missing embeddings/labels

# Connect the labels with the embeddings

import os
import numpy as np
import pandas as pd

# Path to both files
embeddings_dir = '/path/to//embeddings'
label_file_path = '/path/to/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt'

# Load labels file and put in dict
labels_dict = {}
with open(label_file_path, 'r') as file:
  for line in file:
    parts = line.strip().split()
    file_id = parts[1] + ".txt"
    label = parts[-1]
    labels_dict[file_id] = 1 if label == 'bonafide' else 0

print(f"Loaded {len(labels_dict)} labels")

# Ensure the embeddings directory exists
if not os.path.exists(embeddings_dir):
    print(f"Embeddings directory {embeddings_dir} does not exist.")
else:
    print(f"Embeddings directory exists: {embeddings_dir}")

# List files in embeddings directory
embeddings_files = os.listdir(embeddings_dir)
print(f"Total embeddings files: {len(embeddings_files)}")

# Sample embeddings files
print("Sample embeddings files:", embeddings_files[:10])

# Sample label identifiers
sample_labels = list(labels_dict.keys())[:10]
print("Sample label identifiers:", sample_labels)

# Tracking pairing, debugging purposes
successful_loads = 0
failed_loads = 0

# Filter out duplicates and missing files
matched_files = []
embeddings_dict = {}

for filename in embeddings_files:
  filepath = os.path.join(embeddings_dir, filename)
  if filename in labels_dict:
    try:
      embedding = np.loadtxt(filepath)
      embeddings_dict[filename] = embedding
      matched_files.append(filename)
      successful_loads += 1
    except Exception as e:
      print(f"Failed to load {filename}: {e}")
      failed_loads += 1

print(f"Number of matched files: {len(matched_files)}")
print(f"Loaded {successful_loads} embeddings")
print(f"Failed to load {failed_loads} embeddings")

# Filtered out unmatched labels
filtered_labels_dict = {k: v for k, v in labels_dict.items() if k in matched_files}

print(f"Filtered to {len(filtered_labels_dict)} matched labels")

# Create dataframe
data = {
    'embedding': [embeddings_dict[filename] for filename in matched_files],
    'label': [filtered_labels_dict[filename] for filename in matched_files]}


df_dev = pd.DataFrame(data)
print(df_dev.head())

Loaded 24844 labels
Embeddings directory exists: /content/drive/My Drive/THESIS ISABEL/embeddings
Total embeddings files: 25012
Sample embeddings files: ['LA_D_7526656.txt', 'LA_D_1234846.txt', 'LA_D_7640450.txt', 'LA_D_7290012.txt', 'LA_D_9272270.txt', 'LA_D_2517481.txt', 'LA_D_6345960.txt', 'LA_D_8594051.txt', 'LA_D_4092177.txt', 'LA_D_3317455.txt']
Sample label identifiers: ['LA_D_1047731.txt', 'LA_D_1105538.txt', 'LA_D_1125976.txt', 'LA_D_1293230.txt', 'LA_D_1340209.txt', 'LA_D_1376638.txt', 'LA_D_1377420.txt', 'LA_D_1403371.txt', 'LA_D_1590530.txt', 'LA_D_1595036.txt']
Number of matched files: 24720
Loaded 24720 embeddings
Failed to load 0 embeddings
Filtered to 24720 matched labels
                                           embedding  label
0  [-1.3552709817886353, -0.8355754017829895, -0....      0
1  [-0.8626834154129028, -0.9005163908004761, -1....      0
2  [-1.31498122215271, -1.0843329429626465, -0.65...      0
3  [-1.5352778434753418, -1.408457636833191, -0.6...      0
4  

In [ ]:
# save dataframe
file_path = '/path/to/df_dev.csv'
df_dev.to_csv(file_path, index=False)

print(f'DataFrame saved to {file_path}')

DataFrame saved to /content/drive/My Drive/THESIS ISABEL/Dataframes/df_dev.csv


In [ ]:
# save dataframe as pkl
import pandas as pd

pickle_file_path = '/path/to/df_dev.pkl'
df_dev.to_pickle(pickle_file_path)
print(f'DataFrame saved to {pickle_file_path}')

# Load the DataFrame from the pickle file
df_dev_loaded = pd.read_pickle(pickle_file_path)
print(df_dev_loaded.head())

DataFrame saved to /content/drive/My Drive/THESIS ISABEL/Dataframes/df_dev.pkl
                                           embedding  label
0  [-1.3552709817886353, -0.8355754017829895, -0....      0
1  [-0.8626834154129028, -0.9005163908004761, -1....      0
2  [-1.31498122215271, -1.0843329429626465, -0.65...      0
3  [-1.5352778434753418, -1.408457636833191, -0.6...      0
4  [-1.4741795063018799, -1.2784916162490845, -1....      0
